This notebook documents an example using the `"coreir"` mantle target with the `lattice` vendor to target the icestick.

We begin by building a normal `magma` circuit using `mantle` and the `IceStick` board from loam. Notice that we set the `MANTLE_TARGET` environment variable to `coreir`. This directs `mantle` to use the `coreir` implementations of the `mantle` circuits.

In [1]:
from magma import wire
import os
os.environ['MANTLE_TARGET'] = 'coreir'
from mantle import Counter
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.D5.on()

main = icestick.main()

N = 22

counter = Counter(N)
wire(counter.O[N-1], main.D5)

import mantle lattice ice40
import mantle lattice mantle40


To compile to `coreir`, we simply set the `output` parameter to the `magma.compile` command to `"coreir"`.

In [2]:
import magma
magma.compile("build/blink_coreir", main, output="coreir")

We can inspect the generated `.json` file.

In [3]:
with open("build/blink_coreir.json", "r") as coreir_output:
    print(coreir_output.read())

{"top":"global.main",
"namespaces":{
  "global":{
    "modules":{
      "Add22_cout":{
        "type":["Record",[
          ["I0",["Array",22,"BitIn"]],
          ["I1",["Array",22,"BitIn"]],
          ["O",["Array",22,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "bit_const_GND":{
            "modref":"corebit.const",
            "modargs":{"value":["Bool",false]}
          },
          "inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",23]}
          }
        },
        "connections":[
          ["bit_const_GND.out","inst0.in0.22"],
          ["bit_const_GND.out","inst0.in1.22"],
          ["self.COUT","inst0.out.22"],
          ["inst0.in0.0","self.I0.0"],
          ["inst0.in0.1","self.I0.1"],
          ["inst0.in0.10","self.I0.10"],
          ["inst0.in0.11","self.I0.11"],
          ["inst0.in0.12","self.I0.12"],
          ["inst0.in0.13","self.I0.13"],
          ["inst0.in0.14","self.I0.14"],
          ["inst0.in0.1

We can use the `coreir` command line tool to generate verilog.

In [4]:
%%bash
coreir -i build/blink_coreir.json -o build/blink_coreir.v

VJSON for reg: {"definition":"reg [width-1:0] outReg;\nwire real_rst;\nassign real_rst = arst_posedge ? arst : ~arst;\nwire real_clk;\nassign real_clk = clk_posedge ? clk : ~clk;\nalways @(posedge real_clk, posedge real_rst) begin\n  if (real_rst) outReg <= init;\n  else outReg <= in;\nend\nassign out = outReg;","interface":["input clk","input arst","input [width-1:0] in","output [width-1:0] out"],"parameters":["init","arst_posedge","clk_posedge"],"prefix":"coreir_"}
metadata for reg: {"verilog":{"definition":"reg [width-1:0] outReg;\nwire real_rst;\nassign real_rst = arst_posedge ? arst : ~arst;\nwire real_clk;\nassign real_clk = clk_posedge ? clk : ~clk;\nalways @(posedge real_clk, posedge real_rst) begin\n  if (real_rst) outReg <= init;\n  else outReg <= in;\nend\nassign out = outReg;","interface":["input clk","input arst","input [width-1:0] in","output [width-1:0] out"],"parameters":["init","arst_posedge","clk_posedge"],"prefix":"coreir_"}}
Running Runningvpasses
In Run Generators


And now we can inspect the generated verilog from coreir, notice that includes the `verilog` implementations of all the coreir primitives.

In [5]:
with open("build/blink_coreir.v", "r") as coreir_output:
    print(coreir_output.read())

//Module: pullresistor defined externally


module corebit_and (
  input in0,
  input in1,
  output out
);
  assign out = in0 & in1;

endmodule //corebit_and

module corebit_ibuf (
  inout in,
  output out
);
  assign out = in;

endmodule //corebit_ibuf

module coreir_reg #(parameter clk_posedge=1, parameter init=1, parameter width=1) (
  input clk,
  input [width-1:0] in,
  output [width-1:0] out
);
reg [width-1:0] outReg=init;
wire real_clk;
assign real_clk = clk_posedge ? clk : ~clk;
always @(posedge real_clk) begin
  outReg <= in;
end
assign out = outReg;

endmodule //coreir_reg

module corebit_not (
  input in,
  output out
);
  assign out = ~in;

endmodule //corebit_not

module reg_U1 #(parameter init=1) (
  input  clk,
  input [0:0] in,
  output [0:0] out
);
  //Wire declarations for instance 'reg0' (Module coreir_reg)
  wire  reg0__clk;
  wire [0:0] reg0__in;
  wire [0:0] reg0__out;
  coreir_reg #(.clk_posedge(1),.init(init),.width(1)) reg0(
    .clk(reg0__clk),
    .in(reg0__i

In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif blink_coreir.blif' blink_coreir.v
arachne-pnr -q -d 1k -o blink_coreir.txt -p blink_coreir.pcf blink_coreir.blif 
icepack blink_coreir.txt blink_coreir.bin
iceprog blink_coreir.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
